In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 원본 데이터 디렉터리 및 새로운 디렉터리 경로 설정
original_data_dir = "D:\ko_food\한국_음식"
output_dir = "D:\ko_food\한국_음식"

train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")

# train/val 디렉터리 생성
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 데이터 분리 비율 설정
val_split = 0.2  # 20% 데이터를 검증 데이터로 분리

# 각 대분류(category)의 하위 음식별 데이터를 처리
for category in os.listdir(original_data_dir):
    category_path = os.path.join(original_data_dir, category)
    if not os.path.isdir(category_path):
        continue
    
    for food in os.listdir(category_path):
        food_path = os.path.join(category_path, food)
        if not os.path.isdir(food_path):
            continue
        
        # 이미지 파일 수집
        images = [os.path.join(food_path, img) for img in os.listdir(food_path) if img.endswith((".jpg", ".png", ".jpeg"))]
        
        # train/val로 분리
        train_images, val_images = train_test_split(images, test_size=val_split, random_state=42)
        
        # 각 데이터셋으로 복사
        for img_path in train_images:
            dest_dir = os.path.join(train_dir, category, food)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(img_path, dest_dir)
        
        for img_path in val_images:
            dest_dir = os.path.join(val_dir, category, food)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(img_path, dest_dir)

print("데이터가 'train'과 'val'로 성공적으로 분리되었습니다!")


데이터가 'train'과 'val'로 성공적으로 분리되었습니다!


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 1. 커스텀 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.file_paths = []
        self.labels = []
        self.transform = transform

        for class_idx, class_name in enumerate(os.listdir(root_dir)):
            class_dir = os.path.join(root_dir, class_name)
            for food_name in os.listdir(class_dir):
                food_dir = os.path.join(class_dir, food_name)
                for file_name in os.listdir(food_dir):
                    self.file_paths.append(os.path.join(food_dir, file_name))
                    self.labels.append(class_idx)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# 2. 데이터 경로 및 하이퍼파라미터 정의
data_dir = 'D:/ko_food/한국_음식'  # 데이터 경로
image_size = 224
batch_size = 32
num_classes = 150
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = CustomDataset(root_dir=f"{data_dir}/train", transform=transform)
val_dataset = CustomDataset(root_dir=f"{data_dir}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 4. 모델 정의 (ResNet50 기반)
model = models.resnet50(pretrained=True)

# 기존 레이어 고정
for param in model.parameters():
    param.requires_grad = False

# 출력층 수정
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# 5. 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

# 6. 모델 학습 및 검증 함수 정의
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    return model

# 7. 모델 학습
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# 8. 모델 저장
torch.save(trained_model.state_dict(), 'korean_food_model.pth')
print("모델 학습 및 저장 완료!")


c:\Users\human\anaconda3\envs\human-dl-env2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\human\anaconda3\envs\human-dl-env2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------
